In [ ]:
# Install and resolve dependencies

!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers

In [ ]:
# Import AI model from gpt-neo

print("Available models:\n")
print("     -> gpt-neo-2.7B")
print("     -> gpt-neo-1.3B")
print("     -> gpt-neo-350M")
print("     -> gpt-neo-125M")
print("")

from transformers import GPTNeoForCausalLM, GPT2Tokenizer

input = input()
model = GPTNeoForCausalLM.from_pretrained(f'EleutherAI/{input}')
tokenizer = GPT2Tokenizer.from_pretrained(f'EleutherAI/{input}')

In [ ]:
# GPT-Neo Chatbot Agent   (GNC-Agent) ~ Uncommented

class GNC_Agent:
  def __init__(self, max_length, gen_full_sentences, file):
      self.max_length = max_length
      self.gen_full_sentences = gen_full_sentences
      self.memory = ""
      if file: self.ReadMemory(file)

  def CutString(self, string, chars):
    i = 0
    for c in string:
      i += 1
      for s in chars:
        if c == s:
          return string[:i]
    return string

  def ReadMemory(self, file):
    self.memory = ""
    with open(file, 'r') as content:
      self.memory += content.read()
      content.close()

  def WriteMemory(self, file):
    with open(file, 'w') as content:
      content.write(self.memory)
      content.close()

  def Guard(self, prompt):
    if prompt != "":
      if prompt[0] == ',':
        prompt=prompt.replace(',', 'self.')
        eval(prompt)
        return True

  def Undo(self):
    self.memory = self.memory.replace(self.gen_text, "")
    self.gen_text = ""

  def Generate(self, prompt):
    if self.Guard(prompt) == True: return
    self.memory += prompt

    while 1:
      input_ids = tokenizer(prompt, return_tensors="pt").input_ids
      gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=self.max_length, pad_token_id=50256)
      gen_text = self.CutString(tokenizer.batch_decode(gen_tokens)[0].replace(prompt, ''), '.?!')
      if gen_text != tokenizer.batch_decode(gen_tokens)[0].replace(prompt, '') or not self.gen_full_sentences:
        break

    self.memory += gen_text
    return gen_text, self.memory


In [ ]:
agent = GNC_Agent(16, True, False)  # Agent instantiation with base max length set as 16 chars, full sentences generation on (sentences will end with either '.', '?' or '!'), and load file disabled.

print(agent.Generate("You are a student at university.")[0])  # Printing the first returned member of method GNC_Agent.Generate(). Member is gen_text